In [ ]:
from torch.utils.data import DataLoader
import torch
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

# Create test dataloader
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Move model to evaluation mode
model.eval()

# Store predictions and labels
true_labels = []
pred_labels = []
test_texts = []

# Prediction loop
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predictions.cpu().numpy())

        if 'text' in batch:
            test_texts.extend(batch['text'])  # Use original text if available
        else:
            test_texts.extend(["[Text not available]"] * len(labels))

# Confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
label_names = ['Negative (0)', 'Neutral (1)', 'Positive (2)']

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_names, yticklabels=label_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix - Sentiment Analysis')
plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels, target_names=label_names))

# Misclassified Examples
misclassified = []
for text, true, pred in zip(test_texts, true_labels, pred_labels):
    if true != pred:
        misclassified.append((text, true, pred))

if misclassified:
    print("\nMisclassified Examples:")
    for i, (text, true, pred) in enumerate(misclassified[:5]):
        print(f"\nExample {i+1}")
        print(f"Text: {text}")
        print(f"True Label: {true} → Predicted Label: {pred}")
        print("-" * 50)
else:
    print("\n✅ No misclassified examples found. Perfect accuracy on the test set.")
